## Title - Master's Thesis
##### Author : Matthieu Grenier

This notebook provides the code associated with the Master's thesis "Has my value function converged ? A comparison of solution methods".

This notebook was written in Julia 1.11.6. It is an open-source language suited for statistics and Machine Learning, designed for high performance. The language relies on just-in-time (JIT) compilation. This happens to be well suited for dynamic programming. Indeed, dynamic programming workloads rely on tight loops and repeated linear algebra. Julia’s JIT compiler (LLVM) makes plain, readable loops fast without resorting to compiled low-level languages such as C or Fortran. The language is strongly typed, thus the user has the freedom of setting its own types, or let the language automatically infer what suits best. Therefore, value and policy iteration run at compiled speeds.

For a great introduction, see *Statistics with Julia: Fundamentals for Data Science, Machine Learning and Artificial Intelligence* (Nazarathy & Klok, 2021). One can also get familiar with Julia and its application to economics on the [QuantEcon website](https://quantecon.org/).


The code outputs, graphs and tables are reproducible by a third-party (random seeds are fixed). To do so, the reader is welcome to make use of the Julia environment associated with the notebook. Additional informations can be found in the [Pkg.jl documentation](https://pkgdocs.julialang.org/v1/environments/). 

***

### Environment setup

In [4]:
# Code to activate the environment and instantiate dependencies

import Pkg

Pkg.activate(Base.dirname(Base.current_project()) == nothing ? "." : Base.dirname(Base.current_project()))
Pkg.instantiate()

  Activating project at `c:\Users\grnma\OneDrive\Bureau\Master's Thesis\From_EGM_to_NN`


In [5]:
# Required packages for this notebook

using LinearAlgebra, Statistics, Random, Distributions, Printf, BenchmarkTools
using QuantEcon, Interpolations, FastChebInterp, NLsolve, Optim, QuadGK
using Plots, LaTeXStrings
using ForwardDiff

***

### The consumption-saving problem

As per the article of Maliar et al.(2021), we overlook the consumption-saving problem and solve using various methods. A baseline result is obtained using non-linear solvers, and we proceed by comparing 4 options :
- Perturbation methods
- Projection methods
- Deep Learning methods à la Maliar et al.(2021)
- Enhanced DL method


This example is fairly simple, but allows for proper representation of how neural networks can be useful when dealing with kinks in the decision rule. Such kinks arise in most dynamic economic models when a given constraint is binding (for instance the implementation of the ZLB in the HANK model, see ???).


***

### A model to grasp the multiplicity of solution methods

The first model which we consider is fairly simple. It is the same as in Maliar et al.(2021). An agent aims to maximise its consumption under a borrowing constraint. Its program is :

$$
\begin{equation*}
\max_{(c_t), (w_t)}  \sum_{t=0}^\infty \beta^t u(c_t) \quad
\\[5ex]
\text{s. t. } 
\begin{cases}
w_{t+1} = r(w_t - c_t) + e^{y_t} \\
c_t \leq w_t
\end{cases}
\end{equation*}
$$

$y_t$ follows an AR(1) process : $$ y_{t+1} = \rho y_t + \sigma \epsilon_t, \; \epsilon_t \sim \mathcal{N}(0, 1)$$

This program can be extended to all any type of agent : households, firms, governments. Its simplicity allows us to get the gist of the main methods available for economists, with quick and interpretable results.

### Initialization

We iniitialize our model with baseline values. We also provide relevant function that will improve readability and comprehensiveness of the algorithms.

In [ ]:
# The exogenous parameters for the model are specified here
# Explanations of the parameter values can be found in Appendix A

model = (;
    β = 0.9, # Discount factor
    r = 1.04, # Interest rate
    γ = 2.0, # Coefficient of relative risk aversion
    ρ = 0.0, # Autoregressive parameter for income shocks
    σ = 0.1, # Standard deviation of the shock
    ā   = 0.0, # Borrowing limit
    Na = 500, # Number of grid points for assets
    a_min = 0.0, # Minimum asset level
    a_max = 50.0, # Maximum asset level
    )

(β = 0.9, r = 1.04, γ = 2.0, ρ = 0.0, σ = 0.1, ā = 0.0, Na = 500, a_min = 0.0, a_max = 50.0)

In [7]:
# Variants of the utility function

u(c; model = model) = (c^(1 - model.γ) - 1) / (1 - model.γ)
u_prime(c; model = model) = c^(-model.γ)
u_prime_inv(x; model = model) = x^(-1/model.γ);

In [8]:
# Helper functions for the consumption-saving problem
# Eases the understanding of each algorithm detailed below

clamp_to_budget(c, a, y) = min(c, y + model.r*a - model.ā)  # To enforce the borrowing constraint :  a' ≥ ā  ↔ c ≤ w
asset_grid = range(model.a_min, model.a_max; length=model.Na) |> collect;

***

### A comparison between four approaches.

First, let's compute a high-accuracy solution to be our baseline throughout the notebook. We rely on the Endongenous Grid Method (EGM) introduced by Carroll in 2005 (insert link). The main idea of EDG is to generate an exogeneous grid of assets, which yields under invertability of the utility function, associated levels of consumption. By doing so, we avoid a root-finding problem, which can be computationnaly expensive. It shall be categorized as part of the "collocation methods". For small problems, like the one here, EGM is efficient, precise, and robust.

In [ ]:
function egm_solve(model; Nz=11, zgrid=nothing, Pz=nothing, η=1e-8, max_iter=5_000, damp=0.95)
    """
    egm_solve(model; Nz=11, zgrid=nothing, Pz=nothing, η=1e-8, max_iter=5_000, damp=0.95)

    Applies the EGM method to solve our problem, that is to say that it returns the optimal consumption and next-period assets policies.
    - States today : (a, y) with cash-on-hand w = r*a + exp(y)
    - Controls : c (consumption) and a' (next-period assets) with the borrowing limit a' ≥ ā
    - Law of motion : a' chosen on the grid; budget: c + a' = r*a + exp(y)
    - Euler Equation : u'(c) = β * r * E_y'[ u'(c') ]

    Returns `(c, a_next, asset_grid, zgrid, Pz)` where:
    - `c[a_i, z_j]` and `a_next[a_i, z_j]` are policy arrays on the asset and income grids.

    The function supports continuous income grids, by using Tauchen's method to generate a Markov chain for the income process.
    """
    # ---- Unpacking + Grid selection -------------------------------------
    β, r, γ = model.β, model.r, model.γ
    ā       = model.ā
    Na      = model.Na
    a_min   = model.a_min
    a_max   = model.a_max

    # Asset grid (for BOTH a and a′). Start exactly at the borrowing limit.
    asset_grid = range(max(ā, a_min), a_max; length=Na) |> collect

    # Generation of the income process z_t (log-income) using Tauchen's method
    # The generating procedure is only applied if the zgrid or Pz are not provided.²
    if zgrid === nothing || Pz === nothing
        mc = tauchen(Nz, model.ρ, model.σ)              # AR(1): z' = ρ z + σ ε
        zgrid = collect(mc.state_values)                # grid of incomes
        Pz    = Array(mc.p)                             # transition matrix
    else
        Nz = length(zgrid)  # in case user passed a custom grid
    end
    ygrid = exp.(zgrid)                                  # income levels e^{y}

    # ---- Initialization  of the algorithm -------------------------------

    # Policy functions on (a, z) are stored as functions of today's state variables.
    # This means that we will have c[a_i, z_j] and a_next[a_i, z_j] for each asset a_i and income z_j.
    c = Array{Float64}(undef, Na, Nz) # Policies are stored in a matrix form
    a_next = similar(c)

    for j in 1:Nz
        y = ygrid[j]
        # The cash-on-hand is defined for each a as such: w = r*a + y
        w = @. r*asset_grid + y
        # And the agent must respect the borrowing constraint: a′ ≥ ā
        # We set c = 0.97 * w as a stable starting guess for consumption. The 0.97 factor is the same as in QuantEcon's EGM implementation.
        c[:, j] = clamp.(0.97 .* w, 1e-12, w .- ā)
        a_next[:, j] = w .- c[:, j]                       # implied a′ by the budget constraint
    end

    Exp_mu = zeros(Na, Nz) # Exp_mu = Expected marginal utility of consumption

    # ---- EGM iterations -------------------------------------------------
    
    it = 0
    crit = 1.0
    while it < max_iter && crit > η
        it += 1

        # Compute expected marginal utility of c at each a′ and current z
        # We have : Exp_mu[i, j] = E[ u'(c'(a′=asset_grid[i], z')) | z=j ]

        for j in 1:Nz                
            for jp in 1:Nz           # Iteration over the next income state z'
                Exp_mu[:, j] .+= Pz[j, jp] * u_prime.(c[:, jp]; model=model) # Weighted average of u'(c') according to Pz
            end
        end

        # 2b) Euler inversion: u'(c) = β r Exp_mu  ⇒  c*(a′, z) = u'_inv(β r Exp_mu)
        c_star = u_prime_inv.(β * r .* Exp_mu; model=model)  # size (Na, Nz)
        # Numerical safety
        @. c_star = max(c_star, 1e-12)

        # 2c) Endogenous cash-on-hand grid for each (a′, z):
        #     w* = c*(a′, z) + a′
        #     Given income y today, the implied current assets a are:
        #     a = (w* - y) / r
        for j in 1:Nz
            y = ygrid[j]

            w_star = c_star[:, j] .+ asset_grid
            a_of_w = (w_star .- y) ./ r  # "endogenous" current asset grid

            # 2d) Interpolate from (a_of_w, c_star) → c(a, z) on the regular asset_grid.
            #     BUT: for low w (low a), the borrowing constraint binds (a′ = ā).
            #     We handle this by:
            #       • for a below the minimum of a_of_w  ⇒ set c = w - ā (consume all above binding a′=ā)
            #       • for a within the convex hull       ⇒ interpolate linearly
            #       • for a above the maximum            ⇒ extrapolate linearly (rare if grid wide enough)

            # Construct a linear interpolation on the *monotone* region.
            # Ensure strict monotonicity in a_of_w to avoid interpolation errors.
            # If (rare) numerical non-monotonicity, enforce by stable sort + unique.
            idx = sortperm(a_of_w)
            a_of_w_sorted = a_of_w[idx]
            c_star_sorted = c_star[idx, j]

            itp = linear_interpolation(a_of_w_sorted, c_star_sorted; extrapolation_bc=Line())

            # Fill c_new on the regular asset grid
            @inbounds for i in eachindex(asset_grid)
                a = asset_grid[i]
                w = r*a + y

                if a <= first(a_of_w_sorted)   # binding region ⇒ a′=ā ⇒ c = w - ā
                    c_val = max(1e-12, w - ā)
                else
                    # interior / upper region: use interpolated c*
                    c_val = max(1e-12, itp(a))
                    # and never violate the budget c ≤ w - ā numerically
                    c_val = min(c_val, w - ā)
                end

                # damped update for stability
                c[i, j] = damp * c_val + (1 - damp) * c[i, j]
                a_next[i, j] = w - c[i, j]    # implied next assets
            end
        end

        # 2e) Convergence check: change in Euler RHS (cheap, effective)
        crit = maximum(abs.(β * r .* Exp_mu .- u_prime.(c; model=model)))
        # (alternatives: norm of c updates, sup-norm distance of policies, etc.)
        # @show it, crit
    end

    return c, a_next, asset_grid, zgrid, Pz
end

#### Lifetime-reward maximisation

Maliar et al.(2021) show that tackling the lifetime-reward maximisation is equivalent to minimizing the following objective function : 

<br><br>

**Definition 2.4** (All-in-one expectation operator for lifetime reward). *Fix time horizon* $T > 0$, *parametrize a decision rule* $\varphi(\cdot; \theta)$ *and define the distribution of the random variable* $\omega \equiv (m_0, s_0, \epsilon_1, \ldots, \epsilon_T)$. *For given* $\theta$, *lifetime reward associated with the rule* $\varphi(\cdot; \theta)$ *is given by*

$$
\Xi(\theta) = \mathbb{E}_\omega[\xi(\omega; \theta)] \equiv \mathbb{E}_{(m_0, s_0, \epsilon_1, \ldots, \epsilon_T)} \left[ \left. \sum_{t=0}^T \beta^t r(m_t, s_t, \varphi(m_t, s_t; \theta)) \right] \right.,
$$

*where transitions are determined by equations (1), (2) and (3), and* $\xi$ *is an integrand.*

<br><br>


As per Maliar et al.(2021), the estimation procedure is proceeded using deep learning associated with $n \times n'$ Monte-Carlo draws.